In [36]:
import torch 

# 8 bits quantization

f = torch.rand(3,3)*100-50
nbits = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weight_max = torch.max(f)

# 10000. (+1) ==> 100000  (-1) ==>1000.0
fixed_exp = torch.ceil(torch.log2(weight_max/(2**(nbits-1)-1))) #+ 1
print(fixed_exp)

def float2bin(f, fixed_exp, nbits=8):
    print(f"floating pt: ")
    print(f)
    # get sign s = {0,1}
    s = torch.sign(f)
    print(f"sign input: ")
    print(s)
    f = f * s
    print(f"f*s: ")
    print(f)
    s = (s * (-1) + 1.) * 0.5
    print(f"s=(s*-1+1)*0.5: ")
    print(s)
    s= s.unsqueeze(-1)
    print(f"unsqeeze: ")
    print(s)
    f = f/(2**fixed_exp)
    print(f"f/2**fixexp: ")
    print(f)

    # remove decimal places 5.32135 - 0.32135 => 5 
    m = integer2bit(f - f % 1,num_bits = nbits-1)
    print(f"integer2bit from f: ")
    print(m)
    dtype = f.type()
    out = torch.cat([s, m], dim=-1).type(dtype)
    print(f"concaten [s m]: ")
    print(out)
    return out

def integer2bit(integer, num_bits=7):
    dtype = integer.type()
    print("*******input integer: *******")
    print(integer)
    exponent_bits = -torch.arange(-(num_bits - 1), 1).type(dtype)
    print("exponent_bits: ")
    print(exponent_bits)
    exponent_bits = exponent_bits.repeat(integer.shape + (1,))
    print("exponent_bits.repeat(integer.shape + (1,)): ")
    print(exponent_bits)
    out = integer.unsqueeze(-1) / 2 ** exponent_bits
    print("integer.unsqueeze(-1) / 2 ** exponent_bits")
    print(out)

    # remove decimal places 5.32135 - 0.32135, see if elementwise remainder by 2
    out = (out - (out % 1)) % 2
    print("*******out bits: *******")
    print(out)
    return out

def bin2float(b, fixed_exp, nbits=8):
    dtype = torch.float32
    print("This is the input bit b: ")
    print(b)
    s = torch.index_select(b, -1, torch.arange(0, 1).to(device))
    print("This is sign value: ")
    print(s)
    m = torch.index_select(b, -1, torch.arange(1,nbits).to(device))
    print("This is actual value: ")
    print(m)
    out = ((-1) ** s).squeeze(-1).type(dtype)
    print("reprocess the sign array to actually +1,-1: ")
    print(out)
    exponents = -torch.arange(-(nbits - 2.), 1.).to(device)
    print("exponents: ")
    print(exponents)
    exponents = exponents.repeat(b.shape[:-1] + (1,))
    print("exponents.repeat(b.shape[:-1] + (1,))")
    print(exponents)
    e_decimal = torch.sum(m * 2 ** (exponents), dim=-1)
    print("e_decimal: ")
    print(e_decimal)
    out *= e_decimal * 2 ** fixed_exp
    print("Output: ")
    print(out)
    return out


tensor(2.)


In [2]:
bin = float2bin(f,fixed_exp,nbits).to(torch.int8)


floating pt: 
tensor([[-17.5832,  -9.1923,  45.0423],
        [ -9.2020,  46.0960, -46.1917],
        [ 10.1777,  44.2484, -31.8563]])
sign input: 
tensor([[-1., -1.,  1.],
        [-1.,  1., -1.],
        [ 1.,  1., -1.]])
f*s: 
tensor([[17.5832,  9.1923, 45.0423],
        [ 9.2020, 46.0960, 46.1917],
        [10.1777, 44.2484, 31.8563]])
s=(s*-1+1)*0.5: 
tensor([[1., 1., 0.],
        [1., 0., 1.],
        [0., 0., 1.]])
unsqeeze: 
tensor([[[1.],
         [1.],
         [0.]],

        [[1.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [1.]]])
f/2**fixexp: 
tensor([[2.1979, 1.1490, 5.6303],
        [1.1503, 5.7620, 5.7740],
        [1.2722, 5.5310, 3.9820]])
*******input integer: *******
tensor([[2., 1., 5.],
        [1., 5., 5.],
        [1., 5., 3.]])
exponent_bits: 
tensor([2., 1., -0.])
exponent_bits.repeat(integer.shape + (1,)): 
tensor([[[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

        [[2., 1., -0.],
         [2., 1., -0.],
   

In [3]:
after = bin2float(bin,fixed_exp,nbits)
print(after)


This is the input bit b: 
tensor([[[1, 0, 1, 0],
         [1, 0, 0, 1],
         [0, 1, 0, 1]],

        [[1, 0, 0, 1],
         [0, 1, 0, 1],
         [1, 1, 0, 1]],

        [[0, 0, 0, 1],
         [0, 1, 0, 1],
         [1, 0, 1, 1]]], dtype=torch.int8)
This is sign value: 
tensor([[[1],
         [1],
         [0]],

        [[1],
         [0],
         [1]],

        [[0],
         [0],
         [1]]], dtype=torch.int8)
This is actual value: 
tensor([[[0, 1, 0],
         [0, 0, 1],
         [1, 0, 1]],

        [[0, 0, 1],
         [1, 0, 1],
         [1, 0, 1]],

        [[0, 0, 1],
         [1, 0, 1],
         [0, 1, 1]]], dtype=torch.int8)
reprocess the sign array to actually +1,-1: 
tensor([[-1., -1.,  1.],
        [-1.,  1., -1.],
        [ 1.,  1., -1.]])
exponents: 
tensor([2., 1., -0.])
exponents.repeat(b.shape[:-1] + (1,))
tensor([[[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

        [[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

    

In [4]:
after_after = float2bin(after,fixed_exp,nbits).to(torch.int8)
print(after_after)


floating pt: 
tensor([[-16.,  -8.,  40.],
        [ -8.,  40., -40.],
        [  8.,  40., -24.]])
sign input: 
tensor([[-1., -1.,  1.],
        [-1.,  1., -1.],
        [ 1.,  1., -1.]])
f*s: 
tensor([[16.,  8., 40.],
        [ 8., 40., 40.],
        [ 8., 40., 24.]])
s=(s*-1+1)*0.5: 
tensor([[1., 1., 0.],
        [1., 0., 1.],
        [0., 0., 1.]])
unsqeeze: 
tensor([[[1.],
         [1.],
         [0.]],

        [[1.],
         [0.],
         [1.]],

        [[0.],
         [0.],
         [1.]]])
f/2**fixexp: 
tensor([[2., 1., 5.],
        [1., 5., 5.],
        [1., 5., 3.]])
*******input integer: *******
tensor([[2., 1., 5.],
        [1., 5., 5.],
        [1., 5., 3.]])
exponent_bits: 
tensor([2., 1., -0.])
exponent_bits.repeat(integer.shape + (1,)): 
tensor([[[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

        [[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

        [[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]]])
integer.uns

In [ ]:
bin2float(after_after,fixed_exp,nbits)

In [35]:
print(after_after)

a = after_after.shape
print(a)
ans = (after_after).view(-1)

# 0/1 into -1/+1
output_tensor = 2. * ans - 1
print(output_tensor)

print("   ")
print("   ")
print("THIS is the input into the attack...")
print("   ")
print("   ")

# -1/+1 into 0/1
output_tensor = ((output_tensor + 1)/2).to(torch.int8)
print(output_tensor)

# int8 
output_tensor = (output_tensor).view(a)
print(output_tensor)

tensor([[[1, 0, 1, 0],
         [1, 0, 0, 1],
         [0, 1, 0, 1]],

        [[1, 0, 0, 1],
         [0, 1, 0, 1],
         [1, 1, 0, 1]],

        [[0, 0, 0, 1],
         [0, 1, 0, 1],
         [1, 0, 1, 1]]], dtype=torch.int8)
torch.Size([3, 3, 4])
tensor([ 1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,
        -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
        -1.,  1., -1.,  1.,  1., -1.,  1.,  1.])
   
   
THIS is the input into the attack...
   
   
tensor([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1], dtype=torch.int8)
tensor([[[1, 0, 1, 0],
         [1, 0, 0, 1],
         [0, 1, 0, 1]],

        [[1, 0, 0, 1],
         [0, 1, 0, 1],
         [1, 1, 0, 1]],

        [[0, 0, 0, 1],
         [0, 1, 0, 1],
         [1, 0, 1, 1]]], dtype=torch.int8)


In [40]:
def quantize_to_nbits(weight, nbits):

    # get weights exponential 
    weight = layer.weight.data
    weight_max = torch.max(weight)
    fixed_exp = torch.ceil(torch.log2(weight_max/(2**(nbits-1)-1))) 

    # quantize to binary
    binary = float2bin(weight,fixed_exp,nbits).to(torch.int8)

    # return the shape of the binary vector
    bit_shape = binary.shape

    # number of weights * nbits
    all_binary_tensor = binary.view(-1).detach().clone()

    # change the bin tensor from {0,1} to {-1,+1}
    output_tensor = 2. * all_binary_tensor - 1.  

    return output_tensor, bit_shape    # output_tensor = [-1., +1., -1., -1., -1.,....], bit_shape = (Wm x Wn x nbits)


def nbits_to_weight32(layer, nbits, input_tensor, bit_shape):

    # get weights exponential 
    weight = layer.weight.data
    weight_max = torch.max(weight)
    fixed_exp = torch.ceil(torch.log2(weight_max/(2**(nbits-1)-1))) 

    # change the bin tensor from {-1,+1} to {0,1}
    input_tensor = ((input_tensor + 1)/2).to(torch.int8)
    
    # change to output shape 
    output_tensor = input_tensor.view(bit_shape)     

    # and back to floating pt representation
    quantized_f = bin2float(output_tensor,fixed_exp,nbits)

    return quantized_f


In [41]:
out, shape = quantize_to_nbits(f, 4)

floating pt: 
tensor([[ -2.8577,   6.5341, -19.4356],
        [-29.2207,  -1.8568, -36.0054],
        [-24.4522,  15.0978,  -0.4518]])
sign input: 
tensor([[-1.,  1., -1.],
        [-1., -1., -1.],
        [-1.,  1., -1.]])
f*s: 
tensor([[ 2.8577,  6.5341, 19.4356],
        [29.2207,  1.8568, 36.0054],
        [24.4522, 15.0978,  0.4518]])
s=(s*-1+1)*0.5: 
tensor([[1., 0., 1.],
        [1., 1., 1.],
        [1., 0., 1.]])
unsqeeze: 
tensor([[[1.],
         [0.],
         [1.]],

        [[1.],
         [1.],
         [1.]],

        [[1.],
         [0.],
         [1.]]])
f/2**fixexp: 
tensor([[0.7144, 1.6335, 4.8589],
        [7.3052, 0.4642, 9.0014],
        [6.1130, 3.7744, 0.1130]])
*******input integer: *******
tensor([[0., 1., 4.],
        [7., 0., 9.],
        [6., 3., 0.]])
exponent_bits: 
tensor([2., 1., -0.])
exponent_bits.repeat(integer.shape + (1,)): 
tensor([[[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

        [[2., 1., -0.],
         [2., 1., -0.],
   

In [43]:
out.dtype

torch.float32

In [44]:
print(nbits_to_weight32(f, 4, out, shape))

This is the input bit b: 
tensor([[[1, 0, 0, 0],
         [0, 0, 0, 1],
         [1, 1, 0, 0]],

        [[1, 1, 1, 1],
         [1, 0, 0, 0],
         [1, 0, 0, 1]],

        [[1, 1, 1, 0],
         [0, 0, 1, 1],
         [1, 0, 0, 0]]], dtype=torch.int8)
This is sign value: 
tensor([[[1],
         [0],
         [1]],

        [[1],
         [1],
         [1]],

        [[1],
         [0],
         [1]]], dtype=torch.int8)
This is actual value: 
tensor([[[0, 0, 0],
         [0, 0, 1],
         [1, 0, 0]],

        [[1, 1, 1],
         [0, 0, 0],
         [0, 0, 1]],

        [[1, 1, 0],
         [0, 1, 1],
         [0, 0, 0]]], dtype=torch.int8)
reprocess the sign array to actually +1,-1: 
tensor([[-1.,  1., -1.],
        [-1., -1., -1.],
        [-1.,  1., -1.]])
exponents: 
tensor([2., 1., -0.])
exponents.repeat(b.shape[:-1] + (1,))
tensor([[[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

        [[2., 1., -0.],
         [2., 1., -0.],
         [2., 1., -0.]],

    